## Data patching ##
* patching   
 = picture + batching 
* OpenCV (Open Source Computer Vision) 
 = 실시간 이미지 프로세싱에 중점을 둔 프로그래밍 라이브러리  
 = 실시간 이미지 처리 기술  
* OS  
 = 운영체제에서 제공되는 여러 기능을 파이썬에서 수행시켜주는 파이썬 라이브러리(모듈)  
 ex) 파일 복사, 폴더 생성, 폴더 내 파일 목록 구하기

In [1]:
import cv2
import numpy as np
import os

### 1. Setting: 데이터 불러오기 & crop 옵션 설정 ###

* crop의 목적
: 많은 데이터로 정밀한 학습  
* crop의 의미  
: 하나의 이미지를 여러장으로 분할하여 학습 데이터로 사용  

#### 1.1 Set numbers, size of crop: crop할 갯수와 crop 사이즈 결정 #### 

crop_n = 100  
sel_p = 0.0  # ratio of fucused region  
rand_n = int(crop_n * (1 - sel_p))  
crop_wth = 64  
crop_hgt = 64  

#### 1.2 Get image list: 이미지 데이터 가져오기 ####   

    img_list = os.listdir("labeling/img") #labeling/img 안의 이미지에 대한 img_list라는 dir를 만든다.
    name_list = []   
    for img in img_list: # img_list의 각각 img에 os.path.splitext(img)[0]의 원소를 더한다.    
        name_list.append(os.path.splitext(img)[0])
* splitext(): 파일명을 이름과 확장자로 분리후 반환하는 함수

In [2]:
# Set numbers, size of crop
crop_n = 100
sel_p = 0.0  # ratio of fucused region
rand_n = int(crop_n * (1 - sel_p))
crop_wth = 64
crop_hgt = 64

# Get image list
img_list = os.listdir("labeling/img") 
name_list = []
for img in img_list:
    name_list.append(os.path.splitext(img)[0]) 

# # Set focused reference coordinate
# # ref_xy = [[138, 105],
# #           [54, 460],
# #           [938, 154],
# #           [783, 716],
# #           [268, 727],
# #           [758, 765],
# #           [1295, 719],
# #           [685, 660],
# #           [978, 322]]

### 2. Data loading  ###

####  2.1 Load bounding box coordinate & sample image: 바운딩박스와 이미지 데이터 불러오기 ####

for k, img_name in enumerate(name_list):  

    box_coor = np.loadtxt("labeling/bbox/" + img_name + ".txt", delimiter=' ') # bounding box 좌표를 array의 형태로 불러옴  
    img_raw = cv2.imread("labeling/img/" + img_name + ".jpg") # img 파일을 ndarray의 형태로 불러옴   
    img_wth, img_hgt = img_raw.shape[1], img_raw.shape[0] #   
    
* enumerate(name_list): 원소와 원소의 인덱스까지 함께 출력하는 함수
* np.loadtxt(fname(파일명), dtype(데이터형식), delimiter(분리구분기호), skiprows(skip할 행), usecols(불러올 컬럼))
* cv2.imread(파일 이름): image + read, ndarray 형태로 이미지 파일 불러옴

#### 2.2 Convert bounding box coordinate of YOLO form to OpenCV form: 바운딩박스의 좌표 형식 변환, YOLO -> OPENcv  ####

    box_array = np.zeros_like(box_coor)[:, 1:] # box_coor을 복사해서 0열의 값을 0으로 변환
    box_array[:, 2] = box_coor[:, 3] * img_wth # 박스 가로 사이즈
    box_array[:, 3] = box_coor[:, 4] * img_hgt # 박스 세로 사이즈 
    box_array[:, 0] = box_coor[:, 1] * img_wth - box_array[:, 2] / 2 # (box_coor의 1열 = x 좌표 = 가로의 길이) - 앞부분 길이
    box_array[:, 1] = box_coor[:, 2] * img_hgt - box_array[:, 3] / 2 # (box_coor의 2열 = y 좌표 = 세로의 길이) - 윗부분 길이 
    box_array = box_array.round().astype(int)
    
    box_array[:, 2:] = box_array[:, 2:] + 1
    box_array[:, :2] = box_array[:, :2] - 1  
    
* np.zeros_like(shape)[]: array의 [] 부분이 원소가 0인 배열 생성
* np의 shape: 행렬의 차원 
* astype(): 열의 요소의 dtype을 변경하는 함수
* round(): 반올림

#### 2.3 Set reference coordinate of crop: 자를 이미지의 구역 설정해주기(좌표로 설정) ####

    np.random.seed(42 + k * 10) #seed 설정
    crop_ref = np.array(
        [np.random.randint(low=0, high=img_wth - crop_wth, size=rand_n),
         np.random.randint(low=0, high=img_hgt - crop_hgt, size=rand_n)]
    ).T # 

* => ref[x좌표, y좌표]
* np.random.randint(): 균일 분포의 난수 정수 난수 1개 생성
* np.array().T: 배열 전치

#### 2.4 Change bounding box array format to ref. point & end point: 자른 이미지에서 바운딩박스 좌표 설정해주기 & 형태 바꿔주기 ####

    bboxes = box_array.copy() # bboxes 
    bboxes[:, 2] = bboxes[:, 0] + bboxes[:, 2]
    bboxes[:, 3] = bboxes[:, 1] + bboxes[:, 3]

* ndarray[:,n]: 전체 행에 대하여 n번째 열 선택(추출)

#### 2.5 Crop image & bounding box array: 이미지 자르기 실행 ####

    for i, ref in enumerate(crop_ref):
        fname = img_name + "-" + format(ref[0], '04') + format(ref[1], '04')

#### 2.6 Set crop coordinate: 자른 이미지 좌표  ####

        end_cropy = ref[1] + crop_hgt
        end_cropx = ref[0] + crop_wth

* ref[0] = ref[(x좌표, y좌표)]에서 x좌표를 가져옴 
        
#### 2.7 Get cropped image: 자른 이미지 얻기 ####

        img_crop = img_raw[ref[1]:end_cropy, ref[0]:end_cropx, :]
        
#### 2.8 Filter bounding boxes out of the cropped image: 자른 이미지에서 바운딩박스 필터링 하기 ####

        box_crop = bboxes[((bboxes[:, 2] > ref[0]) & (bboxes[:, 3] > ref[1])) & ((bboxes[:, 0] < end_cropx) & (bboxes[:, 1] < end_cropy))]
        
#### 2.9 Change coordinate of the bounding boxes matching to the cropped image: 자른 이미지와 바운딩 박스 좌표 매칭시켜주기 ####
        box_crop[:, [0, 2]] = box_crop[:, [0, 2]] - ref[0]
        box_crop[:, [1, 3]] = box_crop[:, [1, 3]] - ref[1]
        
#### 2.10 Cut off extra regions of the bounding boxes: 바운딩박스 필터링 하기 ####
        box_crop[box_crop[:, 0] < 0, 0] = 0
        box_crop[box_crop[:, 1] < 0, 1] = 0
        box_crop[box_crop[:, 2] > crop_wth, 2] = crop_wth
        box_crop[box_crop[:, 3] > crop_hgt, 3] = crop_hgt
        
#### Change bounding box array format to YOLO style: 바운딩 박스의 데이터 형식을 yolo 형식으로 바꿔주기 ####
        box_yolo = box_crop.copy().astype('float')
        box_yolo[:, 2] = box_yolo[:, 2] - box_yolo[:, 0]  # get box size
        box_yolo[:, 3] = box_yolo[:, 3] - box_yolo[:, 1]
        box_yolo[:, 0] = box_yolo[:, 0] + box_yolo[:, 2] / 2  # centering box coord.
        box_yolo[:, 1] = box_yolo[:, 1] + box_yolo[:, 3] / 2
        box_yolo[:, [0, 2]] = (box_yolo[:, [0, 2]] / crop_wth)
        box_yolo[:, [1, 3]] = (box_yolo[:, [1, 3]] / crop_hgt)
        box_yolo = np.hstack((np.zeros(box_yolo.shape[0])[:, np.newaxis], box_yolo))
        
#### Save cropped images & bounding boxes: 자른 이미지와 바운딩박스 좌표 저장하기 ####

        os.makedirs("data/raw/", exist_ok=True)
        cv2.imwrite("data/raw/" + fname + ".jpg", img_crop)
        np.savetxt("data/raw/" + fname + ".txt", box_yolo,
                   fmt='%i %0.6f %0.6f %0.6f %0.6f')
                   
* os.makedirs("경로/폴더 이름", exist_ok=True): 파일이 없을 때만 파일을 만든다. 
* cv2.imwrite("경로/파일 이름", img_crop): img_crop 파일을 저장
* np.savetxt("경로/파일 이름", box_yolo,
                   fmt='%i %0.6f %0.6f %0.6f %0.6f'): %i(int), %0.6f(소숫점 6자리까지 float의 형태)
    

In [3]:
for k, img_name in enumerate(name_list):
    # Load bounding box coordinate & sample image
    box_coor = np.loadtxt("labeling/bbox/" + img_name + ".txt",
                          delimiter=' ')
    img_raw = cv2.imread("labeling/img/" + img_name + ".jpg")
    
    img_wth, img_hgt = img_raw.shape[1], img_raw.shape[0]
    
    # Convert bounding box coordinate of YOLO form to OpenCV form
    box_array = np.zeros_like(box_coor)[:, 1:]
    box_array[:, 2] = box_coor[:, 3] * img_wth
    box_array[:, 3] = box_coor[:, 4] * img_hgt
    box_array[:, 0] = box_coor[:, 1] * img_wth - box_array[:, 2] / 2 #???
    box_array[:, 1] = box_coor[:, 2] * img_hgt - box_array[:, 3] / 2 #???
    box_array = box_array.round().astype(int)
    
    box_array[:, 2:] = box_array[:, 2:] + 1
    box_array[:, :2] = box_array[:, :2] - 1
    
    #### Cropping Image to Patch ####    
    # Set reference coordinate of crop
    np.random.seed(42 + k * 10)
    crop_ref = np.array(
        [np.random.randint(low=0, high=img_wth - crop_wth, size=rand_n),
         np.random.randint(low=0, high=img_hgt - crop_hgt, size=rand_n)]
    ).T
    
    # # Set the focused reference range to crop
    # ref_sel = []
    # for i in range(len(ref_xy)):
    #     x_min = ref_xy[i][0] - crop_wth
    #     x_max = ref_xy[i][0] + crop_wth
    #     y_min = ref_xy[i][1] - crop_hgt
    #     y_max = ref_xy[i][1] + crop_hgt
    #     
    #     if x_min < 0:
    #         x_max = x_max - x_min
    #         x_min = 0
    #     if x_max > img_wth - crop_wth:
    #         x_min = x_min - (x_max - (img_wth - crop_wth))
    #         x_max = img_wth - crop_wth
    #     if y_min < 0:
    #         y_max = y_max - y_min
    #         y_min = 0
    #     if y_max > img_hgt - crop_hgt:
    #         y_min = y_min - (y_max - (img_hgt - crop_hgt))
    #         y_max = img_hgt - crop_hgt
    #     
    #     ref_s = [[x_min, x_max], [y_min, y_max]]
    #     ref_sel.append(ref_s)
    # 
    # 
    # sel_n = int((crop_n - rand_n) / len(ref_sel))
    # 
    # for i in range(len(ref_sel)):
    #     c_ref = np.array(
    #         [np.random.randint(low=ref_sel[i][0][0],
    #                            high=ref_sel[i][0][1],
    #                            size=sel_n),
    #          np.random.randint(low=ref_sel[i][1][0],
    #                            high=ref_sel[i][1][1],
    #                            size=sel_n)]
    #     ).T
    #     crop_ref = np.vstack((crop_ref, c_ref))
    
    
    # Change bounding box array format to ref. point & end point
    bboxes = box_array.copy()
    bboxes[:, 2] = bboxes[:, 0] + bboxes[:, 2]
    bboxes[:, 3] = bboxes[:, 1] + bboxes[:, 3]
    
    
    # Crop image & bounding box array
    for i, ref in enumerate(crop_ref):
        fname = img_name + "-" + format(ref[0], '04') + format(ref[1], '04')
        # Set crop coordinate
        end_cropy = ref[1] + crop_hgt
        end_cropx = ref[0] + crop_wth
        
        # Get cropped image
        img_crop = img_raw[ref[1]:end_cropy, ref[0]:end_cropx, :]
        
        # Filter bounding boxes out of the cropped image
        box_crop = bboxes[((bboxes[:, 2] > ref[0]) & (bboxes[:, 3] > ref[1])) & ((bboxes[:, 0] < end_cropx) & (bboxes[:, 1] < end_cropy))]
        
        # Change coordinate of the bounding boxes matching to the cropped image
        box_crop[:, [0, 2]] = box_crop[:, [0, 2]] - ref[0]
        box_crop[:, [1, 3]] = box_crop[:, [1, 3]] - ref[1]
        
        # Cut off extra regions of the bounding boxes
        box_crop[box_crop[:, 0] < 0, 0] = 0
        box_crop[box_crop[:, 1] < 0, 1] = 0
        box_crop[box_crop[:, 2] > crop_wth, 2] = crop_wth
        box_crop[box_crop[:, 3] > crop_hgt, 3] = crop_hgt
        
        # Change bounding box array format to YOLO style
        box_yolo = box_crop.copy().astype('float')
        box_yolo[:, 2] = box_yolo[:, 2] - box_yolo[:, 0]  # get box size
        box_yolo[:, 3] = box_yolo[:, 3] - box_yolo[:, 1]
        box_yolo[:, 0] = box_yolo[:, 0] + box_yolo[:, 2] / 2  # centering box coord.
        box_yolo[:, 1] = box_yolo[:, 1] + box_yolo[:, 3] / 2
        box_yolo[:, [0, 2]] = (box_yolo[:, [0, 2]] / crop_wth)
        box_yolo[:, [1, 3]] = (box_yolo[:, [1, 3]] / crop_hgt)
        box_yolo = np.hstack((np.zeros(box_yolo.shape[0])[:, np.newaxis], box_yolo))
        
        # Save cropped images & bounding boxes
        os.makedirs("data/raw/", exist_ok=True)
        cv2.imwrite("data/raw/" + fname + ".jpg", img_crop)
        np.savetxt("data/raw/" + fname + ".txt", box_yolo,
                   fmt='%i %0.6f %0.6f %0.6f %0.6f')